# Signal vs spot

> *This notebook analysizes the signal intensity as a function of the excitation spot.*

## Import software

In [ ]:
from fretbursts import *
sns = init_notebook()

In [ ]:
import os
import pandas as pd
from IPython.display import display

In [ ]:
figure_size = (5, 4)
default_figure = lambda: plt.subplots(figsize=figure_size)
save_figures = True

def savefig(filename, **kwargs):
    if not save_figures:
        return
    import os
    dir_ = 'figures/'
    kwargs_ = dict(dpi=300, bbox_inches='tight')
                   #frameon=True, facecolor='white', transparent=False)
    kwargs_.update(kwargs)
    plt.savefig(dir_ + filename, **kwargs_)

## Data files

In [ ]:
data_dir = './data/multispot/'

In [ ]:
data_dir = os.path.abspath(data_dir) + '/'
assert os.path.exists(data_dir), "Path '%s' does not exist." % data_dir

In [ ]:
from glob import glob
file_list = sorted(glob(data_dir + '*.hdf5'))

In [ ]:
labels = ['7d', '12d', '17d', '22d', '27d', 'DO']
files_dict = {lab: fname for lab, fname in zip(sorted(labels), file_list)}
files_dict

## Plot style

In [ ]:
PLOT_DIR = './figure/'

In [ ]:
import matplotlib as mpl
from cycler import cycler

bmap = sns.color_palette("Set1", 9)
colors = np.array(bmap)[(1,0,2,3,4,8,6,7), :]
mpl.rcParams['axes.prop_cycle'] = cycler('color', colors)
colors_labels = ['blue', 'red', 'green', 'violet', 'orange', 'gray', 'brown', 'pink', ]
for c, cl in zip(colors, colors_labels):
    locals()[cl] = tuple(c) # assign variables with color names
sns.palplot(colors)

In [ ]:
%config InlineBackend.figure_format='retina'  # for hi-dpi displays
sns.set(style='whitegrid', font_scale=1.4, palette=colors)
#palette = ('Paired', 10)
#sns.palplot(sns.color_palette(*palette))
#sns.set_palette(*palette)

# Load DCR

In [ ]:
dcr = pd.DataFrame.from_csv('results/8-pixels_DCR.csv')
dcr.columns = ['D detector', 'A detector']
dcr

In [ ]:
(dcr/1000).plot(lw=3);
plt.ylabel('Dark Counts (kcps)');

In [ ]:
dcr_d = dcr.drop('A detector', axis=1)
dcr_a = dcr.drop('D detector', axis=1)

# Total Rates

In [ ]:
df = pd.DataFrame(index=pd.Index(np.arange(1, 9), name='Spot'), 
                  columns=pd.Index(labels, name='Sample'))
raw_rates_d, raw_rates_a = df.copy(), df.copy()
for k in labels:
    d = loader.photon_hdf5(files_dict[k])
    t = d.acquisition_duration
    raw_rates = np.array([p.size/t for p in d.ph_times_m])
    raw_rates_a[k] = np.array([m.sum()/t for m in d.A_em])
    raw_rates_d[k] = raw_rates - raw_rates_a[k]

In [ ]:
rates_a, rates_d = df.copy(), df.copy()
rates_a.iloc[:,:] = np.array(raw_rates_a) - np.array(dcr_a)
rates_d.iloc[:,:] = np.array(raw_rates_d) - np.array(dcr_d)

In [ ]:
rates_a.plot(lw=3);

In [ ]:
rates_d.plot(lw=3);

In [ ]:
(rates_a / rates_a.max()).plot(lw=3);
plt.ylim(0);
plt.title('A Channel');

In [ ]:
(rates_d / rates_d.max()).plot(lw=3);
plt.ylim(0);
plt.title('D Channel');

In [ ]:
sns.set(font_scale=1.4, palette=colors)
sns.set_style('ticks')

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(14, 5), sharey=True, sharex=True)
plt.subplots_adjust(wspace=0.2)
(rates_d / rates_d.max()).plot(lw=3, marker='o', ms=8, ax=ax[0]);
(rates_a / rates_a.max()).plot(lw=3, marker='o', ms=8, ax=ax[1]);
plt.ylim(0);

ax[0].set_title('D Detector', fontsize=18);
ax[1].set_title('A Detector', fontsize=18);
sns.despine(offset=10, trim=True, ax=ax[0])
sns.despine(offset=10, trim=True, ax=ax[1], left=True)
ax[1].yaxis.set_visible(False)
ax[0].set_ylabel('A.U.')
plt.xlim(0.75, 8.25)
ax[1].legend(loc='lower right', title='Sample')
ax[0].legend_.remove()
fs = 28
ax[0].text(1,0, 'A', fontsize=fs)
ax[1].text(1,0, 'B', fontsize=fs)
savefig('5dsDNA signal vs ch.png')

In [ ]:
rates1 = rates_d.copy()
rates1.columns = ['D %s' % c for c in rates1.columns]
rates2 = rates_a.copy()
rates2.columns = ['A %s' % c for c in rates2.columns]
rates_da = pd.concat([rates1, rates2], axis=1)

In [ ]:
(rates_da / rates_da.max()).plot(lw=3, alpha=0.7);
plt.ylim(0);

In [ ]:
rates = rates_d + rates_a

In [ ]:
rates.plot(lw=3);

In [ ]:
(rates / rates.max()).plot(lw=3);
plt.ylim(0);

# Background rates

In [ ]:
df = pd.DataFrame(index=np.arange(8), columns=labels)
df.index.name = 'CH'
df.columns.name = 'sample'
raw_bg_rates_d, raw_bg_rates_a = df.copy(), df.copy()
for k in labels:
    d = loader.photon_hdf5(files_dict[k])
    d.calc_bg(fun=bg.exp_fit, time_s=30, tail_min_us='auto', F_bg=1.7)
    raw_bg_rates_d[k] = np.asfarray(d.bg_dd).mean(1)
    raw_bg_rates_a[k] = np.asfarray(d.bg_ad).mean(1)

In [ ]:
bg_rates_a, bg_rates_d = df.copy(), df.copy()
bg_rates_a.iloc[:,:] = np.array(raw_bg_rates_a) - np.array(dcr_a)
bg_rates_d.iloc[:,:] = np.array(raw_bg_rates_d) - np.array(dcr_d)

In [ ]:
bg_rates_a.plot(lw=3);
plt.ylim(0, 3500)

In [ ]:
(bg_rates_a / bg_rates_a.max()).plot(lw=3);
plt.ylim(0);

In [ ]:
bg_rates_d.plot(lw=3);

In [ ]:
(bg_rates_d / bg_rates_d.max()).plot(lw=3);
plt.ylim(0);

In [ ]:
bg_rates = bg_rates_d + bg_rates_a

In [ ]:
bg_rates.plot(lw=3);

In [ ]:
(bg_rates / bg_rates.max()).plot(lw=3);
plt.ylim(0);